<a href="https://colab.research.google.com/github/EjbejaranosAI/AI_trends_2023/blob/main/LLama2_easy_use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Llama 2 implementation for free dataset generation

### Install packages and libraries

In [1]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip -q install huggingface_hub
!pip -q install llama-cpp-python==0.1.78
!pip -q install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 807.8/807.8 kB 12.9 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 9.3 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.1/26.1 MB 59.1 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 16.6 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 5.2 MB/s eta 0:00:00
       ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 7.2 MB/s eta 0:00:00
    Creating /tmp/pip-build-env-ls_thejw/overlay/local/bin
    changing mode of /tmp/pip-build-env-ls_thejw/overlay/local/bin/ninja to 755
    changing mode of /tmp/pip-build-env-ls_thejw/overlay/local

### Download model

In [5]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"

In [6]:
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin"

In [3]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [4]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [7]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

In [24]:

prompt = "Write a linear regression in python"
prompt_template=f'''SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: {prompt}


'''

In [25]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [27]:
print(response.keys())
print(response['choices'][0].keys())


dict_keys(['id', 'object', 'created', 'model', 'choices', 'usage'])
dict_keys(['text', 'index', 'logprobs', 'finish_reason'])


In [28]:
print(response['choices'][0]['text'])

SYSTEM: You are a helpful, respectful and honest assistant. Always answer as helpfully.

USER: Write a linear regression in python


SYSTEM: Of course! Here is an example of how you can write a simple linear regression in Python using the scikit-learn library:
```
from sklearn.linear_model import LinearRegression
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv('your_data.csv')

# Create an X and y column for input features and output target, respectively
X = df[['feature1', 'feature2']]
y = df['target']

# Initialize the linear regression model
reg = LinearRegression()

# Train the model using the X and y data
reg.fit(X, y)

# Print the coefficients of the linear regression equation
print(reg.coef_)
```
This code will load your dataset into a Pandas DataFrame, select the input features (in this case, `feature1` and `feature2`) and the output target (`target`), initialize a LinearRegression object, train the model using the X and y data, and print the c